# Lab Bi


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [24]:
df = pd.read_csv('./data/data.csv', sep=';')


In [25]:
df.shape

(7471, 18)

In [26]:
df.sample(5)

,#,index,tconst,titleType,originalTitle,isAdult,startYear,runtimeMinutes,averageRating,numVotes,ordering,region,language,types,attributes,isOriginalTitle,main_genre,secondary_genre
2014,2300,3268,tt12682076,tvSeries,The Wonder Years,0.0,2021.0,21,6.6,4637.0,16.0,IN,en,imdbDisplay,\N,0.0,Comedy,Drama
4018,4318,5764,tt3922798,movie,The Siege of Jadotville,0.0,2016.0,108,7.2,42129.0,15.0,IN,en,imdbDisplay,\N,0.0,Action,Drama
4383,4686,6198,tt5198068,movie,WolfWalkers,0.0,2020.0,103,8.0,35523.0,12.0,IN,en,imdbDisplay,\N,0.0,Action,Adventure
5877,6244,8204,tt0297037,movie,Brown Sugar,0.0,2002.0,109,6.5,9916.0,4.0,CA,en,imdbDisplay,\N,0.0,Comedy,Drama
6851,7227,9522,tt2351310,movie,Gamgi,0.0,2013.0,122,6.6,13992.0,17.0,CA,en,imdbDisplay,\N,0.0,Action,Drama


In [27]:
print(df.attributes.unique() , df.isAdult.unique())

['\\N' nan] [ 0. nan]


In [34]:
df = df.drop(['#', 'index', 'isAdult', 'attributes', 'isOriginalTitle', 'language'], axis=1)

In [29]:
df.titleType.unique()

array(['movie', 'tvSeries', 'MOVIE', 'tvMiniSeries', 'MOvie', 'Movie',
       'tvMovie'], dtype=object)

In [30]:
df.titleType = df.titleType.str.lower()

In [31]:
df.titleType.unique()

array(['movie', 'tvseries', 'tvminiseries', 'tvmovie'], dtype=object)

In [35]:
df.sample(5)

,tconst,titleType,originalTitle,startYear,runtimeMinutes,averageRating,numVotes,ordering,region,types,main_genre,secondary_genre
5185,tt7988304,tvseries,The Young Offenders,2018.0,30,8.0,4393.0,6.0,IN,imdbDisplay,Comedy,No tiene
4254,tt4834206,tvseries,A Series of Unfortunate Events,2017.0,50,7.8,64028.0,48.0,IN,imdbDisplay,Adventure,Comedy
1970,tt1241325,movie,Objectified,2009.0,75,7.0,3678.0,5.0,IN,imdbDisplay,Documentary,No tiene
7406,tt8168186,tvmovie,Jonas,2018.0,82,7.0,6600.0,9.0,CA,imdbDisplay,Drama,Mystery
770,tt0391304,movie,A Lot Like Love,2005.0,107,6.6,70400.0,20.0,IN,imdbDisplay,Comedy,Drama
